# Persistent homology of activations

In this notebook we will show how the layers of a neural network trained on a classifiction task transforms the input data cloud.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

import sys
sys.path.append("..")
from gdeep.neural_nets import sample_nn
from gdeep.neural_nets.utility import train_classification_nn, Layers_list, get_activations
from gdeep.neural_nets.persistence_activations import persistence_diagrams_of_activations, plot_persistence_diagrams

from sklearn import datasets

ModuleNotFoundError: No module named 'utility'

In [ ]:
# Create labeled point cloud data

n_samples = 1000

noisy_circles = datasets.make_circles(n_samples=n_samples, factor=0.3,
                                    noise=.05)


X = noisy_circles[0]
y = noisy_circles[1]

# Tranform to torch.tensor

X_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y).long()

In [ ]:
# Plot data set X, y

label_color = ['#377eb8' if lab else '#ff7f00' for lab in y.tolist()]

plt.scatter(X[:,0],X[:,1],
                color=label_color)

plt.title("Data set with two classes")
plt.show()

In [ ]:
# Build NN
nodes_layer_1 = 32
dropout_p_layer_1 = 0.7
n_epochs = 2000
learning_rate = 0.01

nn_1 = sample_nn.SimpleNN(nodes_layer_1=nodes_layer_1, dropout_p=dropout_p_layer_1)

In [ ]:
# Train NN

train_classification_nn(nn_1, X_tensor, y_tensor)

In [ ]:
# Visualize activations

# Persistence diagrams of layerwise activations for the training set

persistence_diagrams = persistence_diagrams_of_activations(nn_1, X_tensor, homology_dimensions=[0,1])

plot_persistence_diagrams(persistence_diagrams)